In [1]:
import transformers

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    print("Invalid device or cannot modify virtual devices once initialized.")
    pass

In [3]:
tf.__version__

'2.3.0'

In [4]:
train_df = pd.read_json('data/train.jsonl', lines = True)
train_df["img"] = "data/" + train_df["img"]

In [5]:
val_df = pd.read_json('data/dev_unseen.jsonl', lines = True)
val_df["img"] = "data/" + val_df["img"]

In [6]:
img_width = 224
img_height = 224

In [7]:
tokenizer = transformers.DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [8]:
max_len = 128

### Train

In [9]:
def build_model():
    
    encoder = transformers.TFDistilBertModel.from_pretrained("distilbert-base-uncased")
    encoder.trainable = False

    input_img = tf.keras.layers.Input(
        shape = (img_width, img_height, 3), name = "image"
    )
    
    input_ids = tf.keras.layers.Input(name = "input_ids", shape = (max_len,), dtype = tf.int32)

    attention_mask = tf.keras.layers.Input(name = "attention_mask", shape = (max_len,), dtype = tf.int32)

    y = encoder(input_ids, attention_mask = attention_mask)[0]
    y = tf.keras.layers.BatchNormalization()(y)
    
    y = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences = True))(y)
    y = tf.keras.layers.BatchNormalization()(y)

    extractor = tf.keras.applications.EfficientNetB0(include_top = False, input_tensor = input_img, weights = None)
    
    extractor.trainable = True
    
    x = tf.keras.layers.BatchNormalization()(extractor.output)
    x = tf.keras.layers.GlobalMaxPooling2D()(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    y = tf.keras.layers.GlobalMaxPooling1D()(y)
    y = tf.keras.layers.BatchNormalization()(y)
    
    x = tf.keras.layers.concatenate([x, y])
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Flatten()(x)
    
    x = tfa.layers.WeightNormalization(tf.keras.layers.Dense(1024, activation = "relu"))(x)
    
    x = tf.keras.layers.Dropout(0.25)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tfa.layers.WeightNormalization(tf.keras.layers.Dense(256, activation = "relu"))(x)
    
    x = tf.keras.layers.Dropout(0.25)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    out = tfa.layers.WeightNormalization(tf.keras.layers.Dense(1, activation = 'sigmoid'))(x)

    model = tf.keras.models.Model([input_img, input_ids, attention_mask], out)

    return model

In [10]:
model = build_model()
model.compile(optimizer = tfa.optimizers.LazyAdam(0.001), loss = "binary_crossentropy", metrics = ["accuracy", tf.keras.metrics.AUC(name = 'auc')])
model.summary()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'activation_13', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFDistilBertModel for predictions without further training.
Model: "functional_1"
__________________________________________________

In [11]:
def encode_single_sample(img_path, label, text):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels = 3)
    img = tf.image.convert_image_dtype(img, tf.int32)
    img = tf.image.resize(img, [img_height, img_width])
    text = tokenizer(text, return_tensors = 'tf', max_length = 128, padding = 'max_length')
    return {"image": img, "label": label, "text": text}

In [12]:
class HatefulMemes(tf.keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, input_img_paths, label, text):
        self.batch_size = batch_size
        self.input_img_paths = input_img_paths
        self.text = text
        self.label = label

    def __len__(self):
        return len(self.input_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_label = self.label[i : i + self.batch_size]
        batch_text = self.text[i : i + self.batch_size]
        w = np.zeros((self.batch_size,) + (img_height, img_width) + (3,), dtype = "float32")
        x = np.zeros((self.batch_size,max_len,), dtype = "int32")
        y = np.zeros((self.batch_size,max_len,), dtype = "int32")
        z = np.zeros((self.batch_size), dtype = "float32")
        
        for i,j,k,l in zip(range(self.batch_size), batch_input_img_paths, batch_label, batch_text):
            sample = encode_single_sample(j,k,l)
            w[i] = sample["image"].numpy().tolist()
            x[i] = sample["text"]['input_ids'].numpy().tolist()[0]
            y[i] = sample["text"]['attention_mask'].numpy().tolist()[0]
            z[i] = sample["label"]
        
        return [w,x,y], z

In [13]:
train_gen = HatefulMemes(8, train_df["img"].values.tolist(), train_df["label"].values.tolist(), \
                         train_df["text"].values.tolist())
val_gen = HatefulMemes(8, val_df["img"].values.tolist(), val_df["label"].values.tolist(), \
                         val_df["text"].values.tolist())

In [14]:
epochs = 50
early_stopping_patience = 5

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = "val_auc", patience = early_stopping_patience, restore_best_weights = True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_auc', factor = 0.2,
                                                 patience = 5, min_lr = 0.0001)

history = model.fit(train_gen, validation_data = val_gen, epochs = epochs, callbacks = [early_stopping,                                        reduce_lr])

Epoch 1/50
1062/1062 [==============================] - 1829s 2s/step - loss: 0.6799 - accuracy: 0.6250 - auc: 0.6031 - val_loss: 0.8545 - val_accuracy: 0.6045 - val_auc: 0.5745
Epoch 2/50
1062/1062 [==============================] - 1726s 2s/step - loss: 0.6028 - accuracy: 0.6913 - auc: 0.6915 - val_loss: 0.7180 - val_accuracy: 0.6250 - val_auc: 0.5958
Epoch 3/50
1062/1062 [==============================] - 1736s 2s/step - loss: 0.5740 - accuracy: 0.7096 - auc: 0.7259 - val_loss: 0.7591 - val_accuracy: 0.6325 - val_auc: 0.5920
Epoch 4/50
1062/1062 [==============================] - 1764s 2s/step - loss: 0.5454 - accuracy: 0.7351 - auc: 0.7615 - val_loss: 0.8431 - val_accuracy: 0.6138 - val_auc: 0.5888
Epoch 5/50
1062/1062 [==============================] - 1765s 2s/step - loss: 0.5214 - accuracy: 0.7486 - auc: 0.7876 - val_loss: 0.8300 - val_accuracy: 0.6306 - val_auc: 0.5899
Epoch 6/50
1062/1062 [==============================] - 1740s 2s/step - loss: 0.5018 - accuracy: 0.7652 - auc:

KeyboardInterrupt: 

In [15]:
for layer in model.layers:
    if(layer.name.startswith('tf_distil_bert_model')):
        layer.trainable = True
model.compile(optimizer = tfa.optimizers.LazyAdam(0.001), loss = "binary_crossentropy", metrics = ["accuracy", tf.keras.metrics.AUC(name = 'auc')])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image (InputLayer)              [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           image[0][0]                      
__________________________________________________________________________________________________
normalization (Normalization)   (None, 224, 224, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization[0][0]              
_______________________________________________________________________________________

In [16]:
epochs = 50
early_stopping_patience = 5

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = "val_auc", patience = early_stopping_patience, restore_best_weights = True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_auc', factor = 0.2,
                                                 patience = 2, min_lr = 0.0001)

history_new = model.fit(train_gen, validation_data = val_gen, epochs = epochs, callbacks = [early_stopping,                                        reduce_lr])

NameError: name 'model_new' is not defined

### Predict

In [15]:
train_df = pd.read_json('data/test_unseen.jsonl', lines = True)
train_df["img"] = "data/" + train_df["img"]

In [16]:
def val_encode_single_sample(img_path, text):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels = 3)
    img = tf.image.convert_image_dtype(img, tf.int32)
    img = tf.image.resize(img, [img_height, img_width])
    text = tokenizer(text, return_tensors = 'tf', max_length = 128, padding = 'max_length')
    return {"image": img, "text": text}

In [17]:
class ValHatefulMemes(tf.keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, input_img_paths, text):
        self.batch_size = batch_size
        self.input_img_paths = input_img_paths
        self.text = text

    def __len__(self):
        return len(self.input_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_text = self.text[i : i + self.batch_size]
        w = np.zeros((self.batch_size,) + (img_height, img_width) + (3,), dtype = "float32")
        x = np.zeros((self.batch_size,max_len,), dtype = "int32")
        y = np.zeros((self.batch_size,max_len,), dtype = "int32")
        
        for i,j,k in zip(range(self.batch_size), batch_input_img_paths, batch_text):
            sample = val_encode_single_sample(j,k)
            w[i] = sample["image"].numpy().tolist()
            x[i] = sample["text"]['input_ids'].numpy().tolist()[0]
            y[i] = sample["text"]['attention_mask'].numpy().tolist()[0]
        
        return [w,x,y]

In [18]:
pred_gen = ValHatefulMemes(1, train_df["img"].values.tolist(), train_df["text"].values.tolist())

In [19]:
preds = model.predict(pred_gen, verbose = 1)

2000/2000 [==============================] - 288s 144ms/step


In [20]:
def prob2pred(x):
    if x > 0.5:
        return 1
    else:
        return 0

In [21]:
train_df["proba"] = [i[0] for i in preds.tolist()]
train_df["label"] = train_df["proba"].apply(prob2pred)

In [22]:
train_df = train_df.drop(["img","text"], axis = 1)
train_df

,id,proba,label
0,15740,0.165844,0
1,38794,0.808912,1
2,60792,0.127603,0
3,71824,0.341913,0
4,4796,0.063573,0
...,...,...,...
1995,71352,0.941351,1
1996,2164,0.094855,0
1997,3587,0.106884,0
1998,47839,0.231437,0


In [23]:
train_df.to_csv("submission_new.csv", index = False)